In [ ]:
!pip install bs4

In [3]:
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 환경변수 가져오기
load_dotenv()

loader = WebBaseLoader(
    web_path=("https://n.news.naver.com/article/437/0000416134"),
    bs_kwargs=dict(
        # 특정 element만 파싱하도록 제한하는 필터 적용
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class":["newsct_article _article_body", "media_end_head go_trans"]}
        )
    )
)

docs = loader.load()
# print(docs)

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

splits = text_splitter.split_documents(docs)

# 임베딩 생성
embedding = OpenAIEmbeddings()

# 벡터 저장소 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)

# 검색기(retriever) 생성
retriever = vectorstore.as_retriever()

# runtime
# 프롬프트
prompt = PromptTemplate.from_template(
    """
    당신은 질문-답변을 수행하는 AI 어시스턴트입니다.
    주어진 문맥에 검색된 다음 문맥(context)를 사용해 질문에 답해야 합니다.
    만약 주어진 문맥에서 답을 찾을 수 없는 경우, 모른다고 이야기하세요.
    한글로 답변해주세요.

    #Question:
    {question}

    #Context:
    {context}

    #Answer:
    """
)

# LLM 모델
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Chain 구성
news_chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문
answer = news_chain.invoke("APT는 누가 부른 노래야?")
print(answer)

APT는 로제와 브루노 마스가 함께 부른 노래입니다.
